In [1]:
from pandas import DataFrame
import numpy as np

from gfloat import decode_float, round_float
from gfloat.formats import *

%load_ext autoreload
%autoreload 1
%aimport gfloat

In [2]:

# Print all values in a float8 datatype
fmt = format_info_ocp_e5m2
vals = [decode_float(fmt, i) for i in range(256)]
DataFrame(vals).set_index("ival").drop(columns=["fi"])

,fval,valstr,exp,expval,significand,fsignificand,signbit,fclass
ival,,,,,,,,
0,0.000000e+00,0.0,0,-14,0,0.00,0,FloatClass.ZERO
1,1.525879e-05,~1.5258789e-05,0,-14,1,0.25,0,FloatClass.SUBNORMAL
2,3.051758e-05,~3.0517578e-05,0,-14,2,0.50,0,FloatClass.SUBNORMAL
3,4.577637e-05,~4.5776367e-05,0,-14,3,0.75,0,FloatClass.SUBNORMAL
4,6.103516e-05,~6.1035156e-05,1,-14,0,1.00,0,FloatClass.NORMAL
...,...,...,...,...,...,...,...,...
251,-5.734400e+04,-57344.0,30,15,3,1.75,1,FloatClass.NORMAL
252,-inf,-inf,31,16,0,1.00,1,FloatClass.INFINITE
253,NaN,nan,31,16,1,1.25,1,FloatClass.NAN


In [3]:
0b1101 * 2**-4

0.8125

In [21]:
import ml_dtypes
def mlround(v, dty):
  return np.array([v]).astype(dty).item()

def linterp(a,b,t):
  return a * (1 - t) + b * t

fi = format_info_ocp_e5m2
mldtype = ml_dtypes.float8_e5m2

v0 = decode_float(fi, 26).fval
v1 = decode_float(fi, 27).fval
v26p = linterp(v0, v1, .3)
v44 = 44.0
vsub = decode_float(fi, 0x02).fval * 1.125


for i in (0, 1, 2, 3, 7, 15, 23, 35, 47, 53, 123, 126, 127):
  v0 = decode_float(fi, i+0).fval
  v1 = decode_float(fi, i+1).fval
  for alpha in np.arange(0, 1, .3):
    v = linterp(v0, v1, alpha)
    mlval = mlround(v, mldtype)
    print(f'**\n{v=}, {alpha=} {mlval=}')
    val = round_float(fi, v)
    print(val,mlval)
    assert val == mlval

**
v=0.0, alpha=0.0 mlval=0.0
0.0 0.0
**
v=4.57763671875e-06, alpha=0.3 mlval=0.0
fsignificand=0.6, expval=-17
fsignificand=1.2, expval=-18
fsignificand=1.2, expval=-18, bias=15, effective_precision=-2
fsignificand=0.3, expval=-16, bias=15
rounded fsignificand=0 val=0.0
0.0 0.0
**
v=9.1552734375e-06, alpha=0.6 mlval=1.52587890625e-05
fsignificand=0.6, expval=-16
fsignificand=1.2, expval=-17
fsignificand=1.2, expval=-17, bias=15, effective_precision=-1
fsignificand=0.6, expval=-16, bias=15
rounded fsignificand=1 val=1.52587890625e-05
1.52587890625e-05 1.52587890625e-05
**
v=1.3732910156249999e-05, alpha=0.8999999999999999 mlval=1.52587890625e-05
fsignificand=0.8999999999999999, expval=-16
fsignificand=1.7999999999999998, expval=-17
fsignificand=1.7999999999999998, expval=-17, bias=15, effective_precision=-1
fsignificand=0.8999999999999999, expval=-16, bias=15
rounded fsignificand=1 val=1.52587890625e-05
1.52587890625e-05 1.52587890625e-05
**
v=1.52587890625e-05, alpha=0.0 mlval=1.525878

/tmp/ipykernel_31451/3256595189.py:6: RuntimeWarning: invalid value encountered in scalar multiply
  return a * (1 - t) + b * t


AssertionError: 